## Using SQL Database in AWS
* set up product table with agg func from train_df
* set up user table with agg func from train_df
* convert sql tables to pandas dfs
* pickle dfs

In [63]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql
import os
import pickle

In [61]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'instacart_2017_05_01',
 'instacart_eda_local_sql.ipynb',
 'instacart_pandas.ipynb',
 'instacart_train_sql.ipynb',
 'orders.csv',
 'p3_nb1.ipynb']

In [4]:
connection_args = {
    'host': '52.8.86.131',  # You will need to put the PUBLIC IP address of your AWS instance here
    'user': 'ubuntu',    # username on AWS is 'ubuntu'
    'dbname': 'instacart',    # DB that we are connecting to
    'port': 5432             # port opened on AWS
}

connection = pg.connect(**connection_args)
cursor = connection.cursor()


In [3]:
query = "SELECT * FROM train LIMIT 5"

pd_sql.read_sql(query, connection).columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')

### Total Orders by User ID

In [5]:
user_group = """
            SELECT user_id, COUNT(DISTINCT(order_ID)) as total_user_orders
            FROM train
            GROUP BY user_id
            """
pd_sql.read_sql(user_group, connection)

,user_id,total_user_orders
0,1,10
1,2,14
2,5,4
3,7,20
4,8,3
...,...,...
131204,206199,19
131205,206200,23
131206,206203,5
131207,206205,3


### Sum Total Orders by User ID

In [8]:
user_group = """
            SELECT user_id, COUNT(DISTINCT(product_id)) as user_total_products
            FROM train
            GROUP BY user_id
            """
pd_sql.read_sql(user_group, connection)

,user_id,user_total_products
0,1,18
1,2,102
2,5,23
3,7,68
4,8,36
...,...,...
131204,206199,134
131205,206200,95
131206,206203,84
131207,206205,24


### User Average Days Since Order

In [ ]:
user_group = """
            SELECT user_id, AVG(days_since_prior_order) as user_avg_days_since_order
            FROM train
            GROUP BY user_id
            """
pd_sql.read_sql(user_group, connection)

### Fill User_DF SQL Table with User Features from Train SQL Table

In [ ]:
# terminal command for creating new SQL table
'''CREATE TABLE IF NOT EXISTS user_df (
    user_id INT NOT NULL,
    user_total_orders DOUBLE PRECISION NULL,
    user_avg_cartsize DOUBLE PRECISION NULL,
    user_total_products DOUBLE PRECISION NULL,
    user_avg_days_since_prior_order DOUBLE PRECISION NULL,
    user_avg_reorder_ratio DOUBLE PRECISION NOT NULL);'''

In [24]:
fill_user = """
    INSERT INTO user_df (user_id, user_total_orders, user_avg_cartsize, user_total_products, user_avg_days_since_prior_order, user_avg_reorder_ratio)
        SELECT
        user_id,
        COUNT(DISTINCT(order_id)) AS user_total_orders,
        (COUNT(order_id) / COUNT(DISTINCT(order_id))) AS user_avg_cartsize,
        COUNT(DISTINCT(product_id)) AS user_total_products,
        AVG(days_since_prior_order) AS user_avg_days_since_prior_order,
        AVG(reordered) AS user_avg_reorder_ratio
        FROM train
        GROUP BY user_id
        """

In [25]:
cursor.execute(fill_user)

In [57]:
user_query = "SELECT * FROM user_df;"

user_df = pd_sql.read_sql(user_query, connection)

In [27]:
cursor.execute('commit;')

In [65]:
## pickle dataframe for later use
# with open('user_df.pickle', 'wb') as to_write:
#     pickle.dump(user_df, to_write)

## Create Product_df Table with Product Features from Train SQL Table

In [ ]:
# terminal command to create new table
CREATE TABLE IF NOT EXISTS prod_df (
    product_id int NOT NULL,
    prod_total_orders int NOT NULL,
    prod_avg_add_to_cart_order int NOT NULL,
    prod_avg_days_since_prior_order float(2) NOT NULL);

In [28]:
fill_prod = """
    INSERT INTO prod_df (product_id, prod_total_orders, prod_avg_add_to_cart_order, prod_avg_days_since_prior_order)
        SELECT
        product_id,
        COUNT(DISTINCT(order_id)) AS prod_total_orders,
        AVG(add_to_cart_order) AS prod_avg_add_to_cart_order,
        AVG(days_since_prior_order) AS prod_avg_days_since_prior_order
        FROM train
        GROUP BY product_id
        """

In [29]:
cursor.execute(fill_prod)

In [56]:
prod_query = "SELECT * FROM prod_df;"

prod_df = pd_sql.read_sql(prod_query, connection)

In [64]:
## pickle product dataframe for later use
# with open('prod_df.pickle', 'wb') as to_write:
#     pickle.dump(prod_df, to_write)

### Create User_Product SQL Table From Train Table

In [ ]:
# terminal command for creating new SQL table
CREATE TABLE IF NOT EXISTS user_product (
  user_id int NOT NULL,
  product_id int NOT NULL,
  user_prod_total_orders int NOT NULL,
  user_prod_avg_cart_order float(2) NOT NULL,
#     as user_prod_orders_since_prior_order
#     as user_prod_avg_dow
#  user_prod_order_freq double precision NOT NULL);

In [53]:
fill_user_prod = """
INSERT INTO user_product (user_id, product_id,user_prod_total_orders, user_prod_avg_cart_order)
    SELECT
    user_id,
    product_id,
    COUNT(DISTINCT(order_id)) AS user_prod_total_orders,
    AVG(add_to_cart_order) AS user_prod_avg_cart_order   
    FROM train
    GROUP BY user_id, product_id
    """

In [54]:
cursor.execute(fill_user_prod)

In [55]:
user_prod_query = "SELECT * FROM user_product LIMIT 10;"

pd_sql.read_sql(prod_query, connection)

,product_id,prod_total_orders,prod_avg_add_to_cart_order,prod_avg_days_since_prior_order
0,1,1202,6,10.32610
1,2,60,11,9.90000
2,3,193,6,9.63212
3,4,218,10,13.54130
4,5,12,5,11.16670
...,...,...,...,...
49463,49684,7,5,9.00000
49464,49685,31,10,12.06450
49465,49686,81,9,9.51852
49466,49687,9,9,13.22220
